In [ ]:
from bs4 import BeautifulSoup as bs
from bs4 import SoupStrainer
from selenium import webdriver
from selenium.webdriver.common.by import By

import re
import time
import requests
import pandas as pd
import numpy as np

# Notebook for Scraping

After having chosen a destination and fixed dates (the short the period is the less you'll have duplicates), get the link of the main page on Airbnb. 

In [ ]:
# Example for Nice, France :

page = "https://www.airbnb.fr/s/Nice--France/homes?adults=1&refinement_paths%5B%5D=%2Fhomes&checkin=2022-09-12&checkout=2022-09-18&tab_id=home_tab&query=Nice%2C%20France&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=unknown"

<br/>
<br/>
The site returned 300 ads (20 ads spread over 15 pages). The following cell allows you to get the links of the 300 ads. 

In [ ]:
partial_links = []
pages_link = []
hrefs = []
links = []

# Get the link of the 15 pages of results
for y in range(15) :
    pages_link.append(page + f'&items_offset={y*20}&section_offset=3')
    

# Looking for href links in <a> tags.

for z in pages_link :
    selection = SoupStrainer("a")
    b = bs(requests.get(z).content, 'html.parser',parse_only=selection)
    href_tags = b.find_all('a',class_="ln2bl2p dir dir-ltr",href=True)
    hrefs.append(href_tags)
for w in hrefs : 
    for x in w :
        a = x['href']
        partial_links.append(a)
        
        
# Reconstruct the full link of each ad
for i in partial_links :
    v = 'https://www.airbnb.fr'+i
    links.append(v) 

<br/>
<br/>
Check if you have 300 links. 

In [ ]:
len(links)

In order to avoid driver errors or bugs, it is recommended to scrap the links in groups of 50. 100 if your internet connection is stable.

In [ ]:
list1 = []
list1 = links[0:50]

In [ ]:
# Verification
list1

If you need to, test the entire code with 2 or 3 links.

<br/>
Step 1 : Enter in the dictionary the equipment you want to scrap. <br/>
Step 2 : Create a list that contains the equipment that interests you.

In [ ]:
features_dict = {'link':[],'title':[],'price':[],'superhost':[],'grade':[],'nb_comments':[],'nb_travelers':[],'nb_bedrooms':[],'nb_beds':[],
                'nb_bath':[],'city':[],
                'Télévision':[],'Chauffage':[],'Lave-vaisselle':[],'Wifi':[],'Climatisation':[],
                'Entrée privée':[],'Jacuzzi':[],'Barbecue':[],'Équipements de base':[],'Lave-linge':[],'Four à micro-ondes':[], 
                'Réfrigérateur':[],'Cuisinière':[],'Cafetière':[],'Séjours longue durée autorisés':[],'Baignoire':[]
                ,'Accès plage ou bord de mer':[],'Ascenseur':[],'Arrivée autonome':[],
                'Logement de plain-pied':[],'Vue panoramique sur la ville':[],'Vue sur la piscine':[],
                'Vue sur la baie':[],'Vue sur le jardin':[],'Vue sur la plage':[],'Vue sur le parc':[],'Vue sur le port':[],
                'Vue sur la mer':[],'Vue sur la montagne':[],'Oreillers et couvertures supplémentaires':[],
                'Congélateur':[],'Coffre-fort':[],
                'Patio ou balcon : privé(e)':[],'Jardin privé(e), Clôture intégrale':[],'Parking gratuit':[],'Piscine':[],
                'Salle de sport':[],'Animaux acceptés':[],"Clés remises par l'hôte":[] ,'Équipements de cuisine de base':[]}


# Adapt the elements to the language of the airbnb site you are using
equipment = ['Cuisine','Télévision','Chauffage','Coffre-fort',
                'Lave-vaisselle','Wifi','Climatisation','Entrée privée','Jacuzzi','Barbecue','Équipements de base',
                'Lave-linge','Four à micro-ondes', 'Arrivée autonome',
                'Réfrigérateur','Cafetière','Séjours longue durée autorisés','Baignoire',
                'Accès plage ou bord de mer','Ascenseur',
                'Logement de plain-pied','Vue panoramique sur la ville','Vue sur la piscine',
                'Vue sur la baie','Vue sur le jardin','Vue sur la plage','Vue sur le parc','Vue sur le port',
                'Vue sur la mer','Vue sur la montagne','Oreillers et couvertures supplémentaires',
                'Congélateur','Patio ou balcon : privé(e)','Jardin privé(e), Clôture intégrale','Parking gratuit','Piscine',
                'Salle de sport','Animaux acceptés',"Clés remises par l'hôte",'Équipements de cuisine de base']


# The time sleeps are necessary: so as not to have your IP address blocked by the site because of the excessive number of requests
# for a limited period of time AND because they allow pages to take time to load.


for link in list1 :
    
    # Keep the link of the ad
    features_dict['link'].append(link)
    
    # Opening of the driver : I used a Chrome driver
    #driver = webdriver.Chrome(r"C:\Users\sonia\.anaconda\navigator\.anaconda\navigator\scripts\chromedriver.exe")
    time.sleep(5)
    
    # Opening of the ad link in the driver
    driver.get(link)
    time.sleep(5)
    
    
    # Cookie button processing: click 'ok'
    driver.find_elements(By.XPATH,"/html/body/div[5]/div/div/div[1]/div/div/div[3]/section/div[2]/div[2]/button")[0].click()

    
    
    # Scraping main elements :
    
    # Title of the ad
    title_search = driver.find_elements(By.CLASS_NAME, value="_fecoyn4")
    for elem in title_search : 
        features_dict['title'].append(elem.text)

    # Price per night
    prix_search = driver.find_elements(By.CLASS_NAME, value="_tyxjp1")
    for elem in prix_search :
        if elem.text != '' :
            features_dict['price'].append(elem.text)
            

    # Garde of the ad
    grade_search = driver.find_elements(By.CLASS_NAME, value="_17p6nbba")
    for elem in grade_search :
        features_dict['grade'].append(elem.text[0:4])

    # Superhost or not ?
    try:
        superhost = driver.find_element(By.CLASS_NAME, value="_1mhorg9")
        features_dict['superhost'].append(superhost.text)
    except:
        features_dict['superhost'].append('NC')
        
    
    
    # Number of comments
    commentaires_search = driver.find_elements(By.CLASS_NAME, value="_1qx9l5ba")
    for elem in commentaires_search : 
        if elem.text != '' :
            features_dict['nb_comments'].append(elem.text)

            
            
    # Looking for the following elements in a <li> tag 
    li_search = driver.find_elements(By.TAG_NAME, value="li")
    
    # Number of travelers
    features_dict['nb_travelers'].append(li_search[0].text[0:2])
    
    # Number of bedrooms
    if 'St' in li_search[1].text[2:5] : 
        features_dict['nb_bedrooms'].append('0')
    else : features_dict['nb_bedrooms'].append(li_search[1].text[2:5])
    
    # Number of beds
    features_dict['nb_beds'].append(li_search[2].text[2:5])
    
    # Number of bathrooms 
    bathroom = li_search[3].text[2:5]
    features_dict['nb_bath'].append(bathroom)

    
    # We use a regex to get the name of the city
    city_search = driver.find_elements(By.CLASS_NAME, value="_9xiloll")
    for elem in city_search :
        city = elem.text
        regex = '^[^,]*'
        answ = re.search(regex,city)
        features_dict['city'].append(answ.group(0))
        

    time.sleep(5)
    
    # We look for the button "Display the n equipment" and we click on it
    a = driver.find_elements(By.TAG_NAME, value='button')
    for elem in a :
        # Adapt the string to the language of the airbnb site you are using
        if 'équipements' in elem.text :
            elem.click()
            break

    time.sleep(2)
    
    # Looking for the equipment 
    equipements4 = driver.find_elements(By.CLASS_NAME, value="_11jhslp")
    equipements5 = driver.find_elements(By.CLASS_NAME,value='_gw4xx4')
    liste = []
    
    for e in equipements5 :
        liste.append(e.text)
     
    # This loop allows you to search in the list of all the equipment present on the ad.
    # If it finds items from the equipment list created earlier in the code, it keeps them.
    # If it finds other equipment, it does not take them into account at all.
    # Finally, if certain equipment from the equipment list is not mentioned in the ad, it writes NC.
    # And this, even if they are unavailable (crossed out on the ad).
    
    for e in equipment : 
        if e in liste :
            for k in liste :
                if e in k : 
                    if 'Indisponible' in k :
                                    if e in features_dict.keys():
                                        features_dict[e].append('NC')
                                    else : pass
                    else : 
                        if e in features_dict.keys():
                            features_dict[e].append(e)
                        else : pass
        if e not in liste :
                features_dict[e].append('NC')
    
    
    driver.quit()


print(features_dict)

<br/>
To transform the dictionary into a dataframe :

In [ ]:
df = pd.DataFrame.from_dict(features_dict, orient='index')
df = df.transpose()

<br/>
To display all rows and columns :

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
df

<br/>
After 50 announcements or if the algorithm has stopped on the way, it is necessary to store the first complete lines and without error in a dataframe by removing empty or erroneous lines (which arrive at the end).
<br/>
Let the features_dict dictionary reset itself. If we let the add continue the dictionary, we will not have to remove empty or erroneous lines from the dictionary, which is a problem.

In [ ]:
# Example : 
df1 = df.drop(df.index[50:75])

In [ ]:
df1

<br/>
If you need to concat dataframes :

In [ ]:
pd.concat([df1,df2], ignore_index=True)

<br/>
To export your dataframe : 

In [ ]:
df.to_csv('Scrap1.csv',sep='\t')